In [1]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder\
    .master("local[*]")\
    .appName('test')\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/28 10:04:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-29 13:42:10|  2020-01-29 13:48:24|                 N|         1|           7|         193|              3|         0.91|        6.0|  0.0|    0.

In [19]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [4]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [17]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

Rename columns

In [5]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime','pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime','dropoff_datetime') 

In [6]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime','pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime','dropoff_datetime') 

Compare columns for two data frame
Extract the common columns between the two without change the column's order

In [21]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [7]:
set_columns=[]
yellow_columns=set(df_yellow.columns)
for column in df_green.columns:
    if column in yellow_columns:
        set_columns.append(column)

In [8]:
set_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

Add extra column to identify the original tables

In [9]:
from pyspark.sql import functions as F

In [10]:
df_yellow_sel=df_yellow \
    .select(set_columns) \
    .withColumn('service_type',F.lit('yellow'))

In [11]:
df_green_sel=df_green \
    .select(set_columns) \
    .withColumn('service_type',F.lit('green'))

Merge data together

In [12]:
df_trips_data=df_yellow_sel.unionAll(df_green_sel)

In [13]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|      yellow|39649199|
|       green| 2304517|
+------------+--------+



Create temporary view to select the data

In [14]:
df_trips_data.createOrReplaceTempView('trip_data')

In [15]:
spark.sql("""
SELECT * FROM trip_data LIMIT 10;
""").show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|       2|2020-01-08 13:40:30|2020-01-08 13:55:37|                 N|         1|         234|         237|              1|         2.61|       12.0|  0.0|    0.5|      3.06|         0.0|       

In [18]:
df_result=spark.sql("""
select 
PULocationID AS revenue_zone,
date_trunc("month", "pickup_datetime") AS revenue_month, 
service_type, 
SUM(fare_amount) AS revenue_monthly_fare,
SUM(extra) AS revenue_monthly_extra,
SUM(mta_tax) AS revenue_monthly_mta_tax,
SUM(tip_amount) AS revenue_monthly_tip_amount,
SUM(tolls_amount) AS revenue_monthly_tolls_amount,
SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
SUM(total_amount) AS revenue_monthly_total_amount,
avg(passenger_count) AS avg_monthly_passenger_count,
avg(trip_distance) AS avg_monthly_trip_distance
FROM trip_data
GROUP BY 1,2,3
""")

In [19]:
df_result.show()

+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|revenue_zone|revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|avg_monthly_passenger_count|avg_monthly_trip_distance|
+------------+-------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+---------------------------+-------------------------+
|         131|         NULL|      yellow|  121304.70000000004|    4488.400000000001|                 1831.5|        4034.2900000000004|       

In [20]:
df_result.coalesce(1).write.parquet('data/report/revenue/')